# FractalTrader - Live Market Dashboard

Real-time Smart Money Concepts analysis with live data streaming.

**Sprint 2 Deliverable** - Live market monitoring

---

## Features

- ✅ **Real-time price data** (updates every 15s)
- ✅ **Live SMC detection** (automatic pattern updates)
- ✅ **Setup alerts** (visual + audio notifications)
- ✅ **Trade journal** (automatic logging)
- ✅ **24h stability** (error recovery built-in)

---

## Configuration

Customize your live dashboard settings here.

In [ ]:
# === CONFIGURATION ===

SYMBOL = 'BTC'              # Trading symbol (use 'BTC' for Hyperliquid, 'BTC/USDT' for Binance)
DATA_SOURCE = 'hyperliquid' # Data source: 'hyperliquid' or 'binance'
TIMEFRAMES = ['15m', '1h', '4h']  # Timeframes to monitor
UPDATE_INTERVAL = 15        # Update frequency in seconds (min: 5)
MIN_CONFIDENCE = 70         # Minimum confidence to trigger alerts (0-100)
ENABLE_SOUND = True         # Enable audio alerts

print(f"✅ Configuration loaded")
print(f"   Symbol: {SYMBOL}")
print(f"   Source: {DATA_SOURCE}")
print(f"   Update interval: {UPDATE_INTERVAL}s")
print(f"   Alert threshold: {MIN_CONFIDENCE}%")

## Setup

Import libraries and initialize components.

In [ ]:
import sys
sys.path.append('..')  # Add parent directory to path

import logging
from IPython.display import display, HTML
import time
from datetime import datetime

# FractalTrader modules
from notebooks.live_data_stream import LiveIndicatorStream
from notebooks.alert_system import AlertSystem, TradeJournal
from notebooks.setup_detector import SetupDetector, LiveSetupMonitor
from visualization import FractalDashboard

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Suppress verbose logs from libraries
logging.getLogger('hyperliquid').setLevel(logging.WARNING)
logging.getLogger('ccxt').setLevel(logging.WARNING)

print("✅ FractalTrader live dashboard loaded")

## Initialize Components

Create live stream, alert system, and setup detector.

In [ ]:
# Initialize live data stream
stream = LiveIndicatorStream(
    symbol=SYMBOL,
    timeframes=TIMEFRAMES,
    update_interval=UPDATE_INTERVAL,
    source=DATA_SOURCE,
    lookback=500  # Keep 500 candles per timeframe
)

# Initialize alert system
alerts = AlertSystem(
    min_confidence=MIN_CONFIDENCE,
    enable_sound=ENABLE_SOUND
)

# Initialize trade journal
journal = TradeJournal()

# Initialize setup detector
detector = SetupDetector(
    min_confidence=MIN_CONFIDENCE,
    primary_timeframe='1h',
    higher_timeframes=['4h']
)

# Initialize setup monitor
monitor = LiveSetupMonitor(
    detector=detector,
    alert_system=alerts,
    journal=journal
)

# Initialize dashboard
dashboard = FractalDashboard(
    pair=SYMBOL,
    timeframes=TIMEFRAMES
)

print("✅ All components initialized")

## Start Live Stream

Begin streaming live market data.

In [ ]:
# Callback: Check for setups on each update
def on_data_update(data):
    """Called every time new data arrives."""
    try:
        current_price = stream.get_latest_price()
        monitor.check_for_setups(data, SYMBOL, current_price)
    except Exception as e:
        logging.error(f"Setup detection error: {e}")

# Register callback
stream.on_update(on_data_update)

# Start streaming
stream.start()

print("🔴 LIVE - Stream started")
print(f"   Updating every {UPDATE_INTERVAL} seconds")
print(f"   Monitoring {SYMBOL} on {TIMEFRAMES}")
print("\n💡 Run next cell to display live dashboard")

## Live Dashboard

Display real-time charts with SMC overlays.

**Instructions:**
- Charts update automatically every 15 seconds
- New setups trigger visual/audio alerts
- Scroll down to see journal and statistics

**To stop:** Run the "Stop Stream" cell below

In [ ]:
# Display header
display(HTML(f'''
<div style="
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    padding: 20px;
    border-radius: 10px;
    color: white;
    text-align: center;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
    margin: 20px 0;
">
    <h2 style="margin: 0 0 10px 0;">🔴 LIVE Market Dashboard</h2>
    <p style="margin: 0; opacity: 0.9;">{SYMBOL} | {DATA_SOURCE.upper()} | Updates every {UPDATE_INTERVAL}s</p>
</div>
'''))

# Wait for initial data
print("⏳ Waiting for initial data...")
time.sleep(2)

# Get data and display
data = stream.get_data()

if data:
    # Update dashboard data
    for tf, df in data.items():
        dashboard.data[tf] = df
    
    # Detect patterns
    dashboard.detect_patterns()
    
    # Display charts
    dashboard.show(
        height=900,
        show_invalidated=True,
        max_order_blocks=20
    )
    
    # Display current price
    current_price = stream.get_latest_price()
    display(HTML(f'''
    <div style="
        background: #1a1a1a;
        padding: 15px;
        border-radius: 8px;
        color: #10b981;
        text-align: center;
        font-size: 1.5em;
        font-weight: bold;
        margin: 20px 0;
    ">
        Current Price: ${current_price:,.2f}
    </div>
    '''))
    
    print("✅ Dashboard displayed")
    print("\n💡 Charts will auto-update. Alerts will appear below when setups detected.")
else:
    print("❌ No data available yet. Check connection.")

## Live Statistics

View real-time stats and journal entries.

In [ ]:
# Stream statistics
print("📊 Stream Statistics")
print("=" * 50)
print(f"Uptime: {stream.get_uptime():.0f}s")
print(f"Updates: {stream.update_count}")
print(f"Errors: {stream.error_count}")
print(f"Success rate: {(1 - stream.error_count / max(stream.update_count, 1)) * 100:.1f}%")

# Alert statistics
print("\n🔔 Alert Summary")
print("=" * 50)
alert_stats = alerts.get_alert_summary()
for level, count in alert_stats.items():
    if count > 0:
        print(f"{level.upper()}: {count}")

# Journal statistics
print("\n📝 Journal Summary")
print("=" * 50)
journal_stats = journal.get_statistics()
print(f"Total setups: {journal_stats['total_setups']}")
if journal_stats['total_setups'] > 0:
    print(f"Avg confidence: {journal_stats['avg_confidence']:.1f}%")
    print(f"\nBy timeframe:")
    for tf, count in journal_stats['by_timeframe'].items():
        print(f"  {tf}: {count}")
    print(f"\nBy type:")
    for setup_type, count in journal_stats['by_type'].items():
        print(f"  {setup_type}: {count}")

# Recent journal entries
print("\n📋 Recent Setups (Last 5)")
print("=" * 50)
recent = journal.get_recent_entries(5)
if not recent.empty:
    for idx, row in recent.iterrows():
        print(f"\n{idx.strftime('%H:%M:%S')} | {row['timeframe']} | {row['setup_type']}")
        print(f"  Confidence: {row['confidence']:.0f}% | Price: ${row['price']:,.2f}")
else:
    print("No setups detected yet. Keep monitoring...")

## Stop Stream

Run this cell to stop live streaming.

In [ ]:
stream.stop()

print("🛑 Stream stopped")
print(f"\nFinal statistics:")
print(f"  Total updates: {stream.update_count}")
print(f"  Total errors: {stream.error_count}")
print(f"  Uptime: {stream.get_uptime():.0f}s")
print(f"  Setups detected: {journal.get_statistics()['total_setups']}")

## Export Journal

Save journal entries to CSV for analysis.

In [ ]:
# Export to CSV
journal_df = journal.to_dataframe()

if not journal_df.empty:
    filename = f"../data/journal_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    journal_df.to_csv(filename)
    print(f"✅ Journal exported to {filename}")
    print(f"   Total entries: {len(journal_df)}")
else:
    print("⚠ Journal is empty. No data to export.")

---

## 🚀 Next Steps (Sprint 3)

- [ ] Paper trading bot (automated execution)
- [ ] Position management
- [ ] Daily performance reports
- [ ] 7-day testnet validation

---

## 📚 Learn More

- **Live Stream API**: [notebooks/live_data_stream.py](live_data_stream.py)
- **Alert System**: [notebooks/alert_system.py](alert_system.py)
- **Setup Detection**: [notebooks/setup_detector.py](setup_detector.py)
- **Sprint Plan**: [docs/ROADMAP_Q1_2025.md](../docs/ROADMAP_Q1_2025.md)

---

**FractalTrader** - Open source SMC trading system

🚢 Sprint 2 Deliverable (Jan 7 - Jan 20, 2025)